<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Cleaning Sephora dataframe after Scraping with Beautifulsoup


<img src="https://i.insider.com/5838406765edfed40a8b49b1?width=2500&format=jpeg&auto=webp" style="height: 400px; width: 1000px">


Name: Raghad Alharbi

---

- Link to dataset: https://www.kaggle.com/raghadalharbi/all-products-available-on-sephora-website
- Link to starter kernel: https://www.kaggle.com/raghadalharbi/sephora-starter-code
- Second starter kernel: https://www.kaggle.com/kerneler/starter-sephora-website-779abf66-7

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("sephora_all_items_new.csv").drop('Unnamed: 0', axis = 1)
#df.rename(columns = {'brand':'name1', 'name':'brand'}, inplace = True) 
#df.rename(columns = {'name1':'name'}, inplace = True) 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9192 entries, 0 to 9191
Data columns (total 17 columns):
id                        9168 non-null float64
brand                     9168 non-null object
category                  9152 non-null object
name                      9178 non-null object
size                      6007 non-null object
rating                    9177 non-null object
number_of_reviews         9168 non-null object
love                      9163 non-null object
price                     9168 non-null object
value_price               549 non-null object
URL                       9192 non-null object
MarketingFlags            9192 non-null bool
MarketingFlags_content    4382 non-null object
options                   791 non-null object
details                   9154 non-null object
how_to_use                7595 non-null object
ingredients               7538 non-null object
dtypes: bool(1), float64(1), object(15)
memory usage: 1.1+ MB


In [4]:
df.head(2)

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,URL,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients
0,2218774.0,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,SIZE 5 x 0.16oz/5mL•,4 stars,4 reviews,3002 loves,$66.00,($75.00 value),https://www.sephora.com/product/blu-mediterran...,True,online only,NaN,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...
1,2044816.0,Acqua Di Parma,Cologne,Colonia,SIZE 0.7 oz/ 20 mL•,4.5 stars,76 reviews,2.7K loves,$66.00,NaN,https://www.sephora.com/product/colonia-P16360...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,NaN,NaN


In [5]:
df = df[df['price'].notna()]
df['name'].isna().sum()

0

In [6]:
df.loc[df['name']== 'Moonchild Glow Kit', 'number_of_reviews'] = '573'

In [7]:
df['rating']= df['rating'].fillna('0').str.split(' stars', expand=True)[0].str.replace('star', '' ).replace('No', 0 ).astype(float)

In [8]:
df['number_of_reviews']= df['number_of_reviews'].str.split('reviews', expand=True)[0].str.replace('review', '' ).replace(' loves', '' ).fillna('0').replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [9]:
df['love']= df['love'].str.split('loves', expand=True)[0]
df['love']= df['love'].fillna('0').replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
df['price']= df['price'].fillna('0').str.replace('$', '').astype(float)
df['value_price']= df['value_price'].fillna(0).str.replace('\(', '').str.replace('value\)', '').str.replace('$', '').astype(float)
df['size'] = df['size'].str.split('SIZE ', expand=True)[1].str.split('•', expand=True)[0]

In [10]:
MarketingFlags = ['online only', 'exclusive','limited edition', 'limited time offer' ]
df['MarketingFlags_content'] = df['MarketingFlags_content'].fillna('0')
df['online_only'] = np.where((df['MarketingFlags_content'].str.contains('online only')) , 1, 0)
df['exclusive'] = np.where((df['MarketingFlags_content'].str.contains('exclusive')) , 1, 0)
df['limited_edition'] = np.where((df['MarketingFlags_content'].str.contains('limited edition')) , 1, 0)
df['limited_time_offer'] = np.where((df['MarketingFlags_content'].str.contains('limited time offer')) , 1, 0)
df['MarketingFlags_content'].unique()

array(['online only', 'exclusive · online only', '0',
       'limited edition · exclusive · online only',
       'limited edition · online only', 'exclusive',
       'limited edition · exclusive', 'limited edition',
       'limited time offer',
       'limited time offer · limited edition · exclusive'], dtype=object)

In [11]:
df[df['MarketingFlags']==False]

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
11,1417633.0,Acqua Di Parma,Perfume,Bergamotto di Calabria,2.5 oz/ 74 mL,4.5,16,1900,120.0,NaN,...,False,0,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,The land of Calabria is strong and intense. On...,NaN,NaN,0,0,0,0
53,1736065.0,AERIN,Lotions & Oils,Waterlily Sun Body Cream,1 oz / 30 mL,5.0,4,961,15.0,NaN,...,False,0,NaN,What it is:\n A richly luxurious and deeply re...,NaN,NaN,0,0,0,0
57,2189975.0,Aether Beauty,Eye Palettes,Solstice Eyeshadow Palette,NaN,5.0,116,6100,58.0,NaN,...,False,0,NaN,What it is: A palette of 12 golden hour shades...,Suggested Usage:-Use your favorite brushes to ...,-Organic Coconut Oil: Creates a buttery smoot...,0,0,0,0
58,2189959.0,Aether Beauty,Eye Palettes,Rose Quartz Crystal Gemstone Eyeshadow Palette,NaN,5.0,170,8200,58.0,NaN,...,False,0,NaN,What it is: A palette with 12 shades infused w...,Suggested Usage:-Use your favorite brushes to ...,-Organic and Fair Trade** Coconut Oil: Creates...,0,0,0,0
59,2325223.0,Aether Beauty,Highlighter,Supernova Crushed Diamond Highlighter,0.37 fl oz/ 10.5 g,5.0,59,3900,38.0,NaN,...,False,0,NaN,What it is: A super-soft- ethereal glow highli...,Suggested Usage:-Can be used on cheeks- eyes- ...,-Ethically Sourced Diamond Powder: Helps to p...,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9096,2164150.0,SEPHORA COLLECTION,Hair Products,Hold It Together: No-Slip Bobby Pins,NaN,4.5,11,2300,5.0,NaN,...,False,0,NaN,What it is: A set of 45 non-slip bobby pins in...,NaN,NaN,0,0,0,0
9144,2164135.0,SEPHORA COLLECTION,Hair Accessories,Hold It Together: Snag-Free Elastics,NaN,1.5,82,2100,5.0,NaN,...,False,0,NaN,What it is: A set of 100 hair elastics in an a...,NaN,NaN,0,0,0,0
9153,2164259.0,SEPHORA COLLECTION,Sponges & Applicators,Mint Chip Beauty Sponge By Vera Mona,NaN,3.0,19,1600,14.0,NaN,...,False,0,NaN,What it is: A Makeup Sponge collaboration with...,Suggested Usage:-Wet Mint Chip sponge with wat...,NaN,0,0,0,0
9190,50.0,SEPHORA COLLECTION,NaN,Gift Card,NaN,5.0,46,0,50.0,NaN,...,False,0,NaN,What it is:- Available in denominations of $10...,NaN,NaN,0,0,0,0


In [12]:
df.isna().sum()

id                           0
brand                        0
category                    16
name                         0
size                      3161
rating                       0
number_of_reviews            0
love                         0
price                        0
value_price               8619
URL                          0
MarketingFlags               0
MarketingFlags_content       0
options                   8377
details                     14
how_to_use                1573
ingredients               1630
online_only                  0
exclusive                    0
limited_edition              0
limited_time_offer           0
dtype: int64

In [18]:
df['category'] = df['category'].fillna('no category')
df['size'] = df['size'].fillna('no size')
#df['rating'] = df['rating'].fillna('0')
#df['love'] = df['love'].fillna('0')
df['value_price'] = df['value_price'].fillna(df['price'])
df['options'] = df['options'].fillna('no options')
df['details'] = df['details'].fillna('no details')
df['how_to_use'] = df['how_to_use'].fillna('no instructions')
df['ingredients'] = df['ingredients'].fillna('unknown')
df['id'] = df['id'].astype(int)

In [19]:
df.isna().sum()

id                        0
brand                     0
category                  0
name                      0
size                      0
rating                    0
number_of_reviews         0
love                      0
price                     0
value_price               0
URL                       0
MarketingFlags            0
MarketingFlags_content    0
options                   0
details                   0
how_to_use                0
ingredients               0
online_only               0
exclusive                 0
limited_edition           0
limited_time_offer        0
dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9168 entries, 0 to 9191
Data columns (total 21 columns):
id                        9168 non-null int64
brand                     9168 non-null object
category                  9168 non-null object
name                      9168 non-null object
size                      9168 non-null object
rating                    9168 non-null float64
number_of_reviews         9168 non-null int64
love                      9168 non-null int64
price                     9168 non-null float64
value_price               9168 non-null float64
URL                       9168 non-null object
MarketingFlags            9168 non-null bool
MarketingFlags_content    9168 non-null object
options                   9168 non-null object
details                   9168 non-null object
how_to_use                9168 non-null object
ingredients               9168 non-null object
online_only               9168 non-null int64
exclusive                 9168 non-null int64
limi

In [21]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


In [22]:
df.to_csv('sephora_website_dataset.csv',index=False)